# Market-Basket Analysis

From a given set of items I and a set of transactions T, count the number of maps X $\to$ Y such that
   $X,Y \in I$, $X\cap Y$=$\phi$,
-   If X is bought, then it is very likely that Y is also bought.
-   i.e., if $X\subseteq t_j$, then it is very likely that $Y\subseteq t_j$.

Supermarkets use this analysis very frequently. By finding such patterns, they can keep these frequently bought items next to each other in the shop.
This will increase their productivity as customers will be even more eager to buy both the items simultaneously.

As an example, there is the diaper-beer legend. Wal-mart once made such an analysis on the items which were bought in their stores by customers. They found out that, usually on Fridays the dads who bought diapers also tend to buy beers. This was a very unexpected finding. At first there seems to be no connection between diapers and beer.
It was later concluded that the men who bought diapers on Fridays, to give themselves a reward to work on weekdays and since their was no time left to go to the beer shop, they bought the beers and took them home.

Though this technique was initially developed for shop owners, it can also be applied in other cases too where we want to find patterns of set of events that occured together frequently. For example, from the list of weak concepts for each student, we can tell that a student having difficulty in understanding concept A also tend to have difficulty in understanding concept B.

This technique is difficult to implement when one needs to make complex machine learning programs such as Decision Trees, Bayesian Classifiers or Deep Learning Models. But on the positive side, this technique has a plus point of providing a high level explainability that other complex models lack. As we will see, we can even make a classification model with certain datasets with this technique, though not in the way we are used to, but giving us the associations between attributes and their corresponding target class.

## Support and confidence

Confidence level $\chi$

-   How frequently does X $\subseteq$ $t_j$ imply Y $\subseteq$ $t_j$
$$\frac{(X\cup Y).count}{X.count}\geq \chi$$

Support level $\sigma$

-   How significant is this pattern
$$\frac{(X\cup Y).count}{M}\geq\sigma$$

In [1]:
# Setting the support and confidence level.
# We can change them as our requirements.

support = 0.01
confidence = 0.40

### Uploading a dataset to start with

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv("C:\\Users\\Hp\\Downloads\\groceries - groceries.csv")

In [4]:
dataset.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Note:** We can download the above dataset from Kaggle using this link - https://www.kaggle.com/datasets/irfanasrullah/groceries?select=groceries.csv

First we need some preprocessing

In [5]:
dataset = dataset.drop(columns=["Item(s)"])

In [6]:
M, _ = dataset.shape

In [7]:
M

9835

In [8]:
# Creating a dictionary of all the transactions

T = {}

for i in range(M):
    T[i] = list(dataset.iloc[i,:].dropna())

In [9]:
T[0]

['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']

In [10]:
# Now creating a list of all the available items

I = set()

for i in range(M):
    I = I.union(set(list(dataset.iloc[i,:].dropna())))
I = list(I)

In [11]:
N = len(I)

Now that our list of all items and the transactions are made, we are ready to begin with our project.

## Finding the frequent itemsets

First we have to find the frequent itemsets (sets of items with enough support).

i.e., we have to find the items Z$\subseteq$ I such that $Z.count\geq \sigma . M$

I = $\{i_1,i_2, \dots, i_N\}$

T = $\{t_1,t_2,\dots, t_M\}$ where $t_j\subset I$ for all j

To find Z $\subset$ I such that Z.count $>=$ $\sigma$ .M

### 1) The Naive Strategy

In [12]:
# This code should not be executed due to exponential time complexity.

# subsets = [[]] # Returns all subsets of I (The power set of I)
# for el in I:
#     subsets += [s+[el] for s in subsets]

In [13]:
def naive_frequent_itemsets(q):
    frequent = []
    for Z in subsets:
        c=0
        for t in T:
            if Z in t:
                c+=1
        if c>=q*M:
            frequent.append(Z)
    return frequent

But this causes to produce all the subsets of I ($2^N$ many of them) and then maintain $2^N$ many counters. This is computationally very expensive. Usually we will be having a very large number of items to check through.

Is there any better strategy to count the frequent itemsets ?

### 2) The Apriori Algorithm

Lets assume a bound on $|t_j|$ for each $t_j\in M$ (there must be a limit to the number of items a customer buys).
Lets say the threshold is 10

Say N = |I| = $10^6$ and M = |T| = $10^9$, $\sigma = 0.01$ ($1 \%$)

So we only have to count $\sum_{i=1}^{10} \binom{10^6}{i}$ subsets of I.

###### The first important strategy

If $Z\subseteq I$ is a frequent itemset, i.e.,
$$Z.count \geq \sigma . M$$
then each element of Z is also a frequent itemset (a singleton set).

Hence if we count all the frequent singleton itemsets, we can build on them to find all the frequent itemsets.

Since threshold on each transaction is 10 and there are total $10^9$ transactions, there will be at most $10^{10}$ items in $T$.

Also, an item $\{x\}$ that is frequent will appear in at least $10^9 \times 0.01 = {10}^7$ transactions.

Hence there will be at most $10^{10}/10^7 = 1000$ items that are frequent.

#### Apriori Algorithm

If Z is a frequent itemset, then each $Y\subseteq Z$ will also be frequent.

Also, if Z is not frequent, then no Y such that $Z\subseteq Y$ can be frequent.

If $\{x,y\}$ are frequent, then so are $\{x\}$ and $\{y\}$

We can build on this inductively starting from singleton frequent itemsets.

$F_1$ = collection of frequent itemsets $\{x\}$ in I

$C_2$ = all tuples from elements of $F_1$

$F_2$ = all the frequent itemsets from $C_2$

$C_3$ = $\{\{x,y,z\} \text{ }|\text{ } \{x,y\}, \text{ }\{y,z\},\text{ }\{x,z\}\text{ }\in F_2 \}$

$F_3$ = $\{l\in C_3\text{ }| \text{ } l\text{.count} \geq \sigma.M\}$

$C_k$ = subsets of size k, such that each of their (k-1) subsets are in $F_{k-1}$

And so on, we can work for all such frequent itemsets.

When to Stop ?
     When there are no items to check or when we have reached the threshold (like 10 in our case).

But even generating C_k is also expensive, since it has to check for al the k-1 subsets in $F_{k-1}$

If $C_k$ $\subseteq$ $C_k^{'}$, then this $C_k^{'}$ will also work to find $F_{k+1}$



What we will do is first decide a numerical indexing for all the items. Any indexing will do.

Now we will be able to arrange the items $$i_1<i_2<\dots<i_N$$

From $F_k$, list the elements in ascending order.

Merge two k subsets from this $F_{k}$ if their first first k-1 elements are same and they only differ in their last element.

X = $\{i_1, i_2, \dots, i_{k-1}, i_k\}$

$X^{'}$ = $\{i_1,i_2, \dots, i_{k-1}, i_k^{'}\}$

Assuming $i_k < i_k^{'}$

Merge($X,X^{'}$) = $\{i_1,i_2,\dots, i_{k-1}, i_k, i_k^{'}\}$

$C_{k+1}^{'} $ = $\{Merge(X,X^{'})\text{ }| \text{ } X,X^{'}\in F_k\}$

Now we claim that $C_k\subseteq C_k^{'}$

Suppose that $w = \{i_1,i_2,\dots, i_{k-1}, i_k\}\in C_k$

Then $w_1 = \{i_1,i_2,\dots, i_{k-1}\}\in F_{k-1}$

Also, $w_2 = \{i_1,i_2,\dots, i_{k-2}, i_k\}\in F_{k-1}$
$\Rightarrow Merge(w_1,w_2) = w\in C_k^{'}$

Note that |$C_k^{'}$| $\geq $ |$C_k$| so finding $F_{k+1}$ from $C_k^{'}$ will take more steps. But also we are able to generate $C_k^{'}$ very efficiently than generating $C_k$. This compensates the earlier complexity.

Let us find the largets size of a transaction in our dataset

In [14]:
# Find the size of largest transaction
m = 0
for i in T.keys():
    a = len(T[i])
    if a > m:
        m = a
m

32

Before we move on with our project, we will first assign each item a unique number. Then we make two dictionaries to convert between names and unique numbers.

In [15]:
name_to_num = dict(zip(I, range(N)))
num_to_name = dict(zip(range(N), I))

This is done so that our program uses digits to compute and not strings of characters. Working with integers is easier than working with strings because they are less expensive in both memory and complexity. By the two dictionaries above, we can change back and forth between both notations whenever we feel like.

Ideally, we will do all our computations with numbers and then change them back to their corresponding names only at the time of output.

In [16]:
T_num = dict()
for key in T.keys():
    T_num[key] = [name_to_num[x] for x in T[key]]

We will make a count function to count the number of transactions a particular itemset is a subset of.

In [17]:
def count(Z):
    c = 0
    for i in range(M):
        if set(Z).issubset(set(T_num[i])):
            c += 1
    return c

Now let's code what we discussed everything above and find the frequent itemsets

In [18]:
frequent = {}
C = {}

# Frequent itemsets of size 1
C[1] = [[t] for t in list(range(N))]
frequent[1] = [t for t in C[1] if count(t) >= M*support]


# Frequent itemsets of size >= 2
k = 2
while(k <= m and len(frequent[k-1]) >= 2):
    C[k] = []
    x = frequent[k-1]
    
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            if k == 1:
                if x[i] < x[j]:
                    C[2].append([x[i], x[j]])
                else:
                    C[2].append([x[j], x[i]])
            else:
                if x[i][:-1] == x[j][:-1]:
                    if x[i][-1] < x[j][-1]:
                        C[k].append(x[i][:-1] + [x[i][-1]] + [x[j][-1]])
                    else:
                        C[k].append(x[i][:-1] + [x[j][-1]] + [x[i][-1]])
    frequent[k] = [t for t in C[k] if count(t) >= M*support]
    k += 1

del frequent[k-1]

In [19]:
frequent.keys()

dict_keys([1, 2, 3])

This finishes our apriori algorithm.

Now we go to the more challenging part - the association rule mining.

### Association rules

We have to find such $X\cup Y$ frequent itemsets such that $X\to Y$

i.e., $$\frac{(X\cup Y).count}{X.count}\geq \chi$$

First lets go with the easy and most widely used case - the one to one case

In [20]:
A = frequent[2]

In [21]:
q = 1
for f in A:
    x,y = f[0],f[1]
    if count(f) >= confidence * count([x]):
        print(str(q) + ') ' + str(num_to_name[x]) +' -> ' + str(num_to_name[y]) + ', with support = ' + '{:.2f}'.format(count(f)/M * 100) + '% and confidence = ' + '{:.2f}'.format(count(f)/count([x]) * 100) + '%')
        q += 1
    if count(f) >= confidence * count([y]):
        print(str(q) + ') ' + str(num_to_name[y]) +' -> ' + str(num_to_name[x]) + ', with support = ' + '{:.2f}'.format(count(f)/M * 100) + '% and confidence = ' + '{:.2f}'.format(count(f)/count([y]) * 100) + '%')
        q += 1

1) butter -> whole milk, with support = 2.76% and confidence = 49.72%
2) yogurt -> whole milk, with support = 5.60% and confidence = 40.16%
3) oil -> whole milk, with support = 1.13% and confidence = 40.22%
4) frozen vegetables -> whole milk, with support = 2.04% and confidence = 42.49%
5) margarine -> whole milk, with support = 2.42% and confidence = 41.32%
6) hard cheese -> whole milk, with support = 1.01% and confidence = 41.08%
7) domestic eggs -> whole milk, with support = 3.00% and confidence = 47.28%
8) hamburger meat -> whole milk, with support = 1.47% and confidence = 44.34%
9) chicken -> whole milk, with support = 1.76% and confidence = 41.00%
10) whipped/sour cream -> whole milk, with support = 3.22% and confidence = 44.96%
11) white bread -> whole milk, with support = 1.71% and confidence = 40.58%
12) curd -> whole milk, with support = 2.61% and confidence = 49.05%
13) root vegetables -> whole milk, with support = 4.89% and confidence = 44.87%
14) beef -> whole milk, with s

The output obtained above gives us a lot of insights about the grocery store and its customers. For instance, from the 6th output above, nearly 50% of customers who bought butter also bought whole milk.

There are many other such insights in the above output. We can look at the more specific ones my tweaking the values of support and confidence.

It should be noted that this is a very special case of Machine Learning, where the machine finds patterns in the data and displays them to its users so that they can get new insights from them.

There is another case where we can apply this method.

Suppose we are given a bunch of lists with some common categrical values and corresponding to each list there is a target category in which that list belongs. We want to understand what kinds of words are needed together to classify them into a particular category, i.e., we want to learn the relation between list values and their corresponding categories.

In this case, we can tweak the algorithms above and make use of the `many to one` case, having maps from subsets of lists to the categories.

This is where the fun part of this algorithm begins. But since this project have already been very long, we will just construct the general `many to one` case for the above dataset and call it a day. We will be doing the other case of this algorithm in a future project.

In [22]:
B = []
for i in frequent.keys():
    if i >=2:
        B = B + frequent[i]

In [23]:
q = 1
for f in B:
    for c in range(len(f)):
        r = f[c]
        l = f[:c] + f[c+1:]
        if count(f) >= confidence * count(l):
            left = []
            for i in l:
                left.append(num_to_name[i])
            print(str(q) + ') ' + str(left) +' -> ' + str(num_to_name[r]) + ', with support = ' + '{:.2f}'.format(count(f)/M * 100) + '% and confidence = ' + '{:.2f}'.format(count(f)/count(l) * 100) + '%')
            q += 1

1) ['butter'] -> whole milk, with support = 2.76% and confidence = 49.72%
2) ['yogurt'] -> whole milk, with support = 5.60% and confidence = 40.16%
3) ['oil'] -> whole milk, with support = 1.13% and confidence = 40.22%
4) ['frozen vegetables'] -> whole milk, with support = 2.04% and confidence = 42.49%
5) ['margarine'] -> whole milk, with support = 2.42% and confidence = 41.32%
6) ['hard cheese'] -> whole milk, with support = 1.01% and confidence = 41.08%
7) ['domestic eggs'] -> whole milk, with support = 3.00% and confidence = 47.28%
8) ['hamburger meat'] -> whole milk, with support = 1.47% and confidence = 44.34%
9) ['chicken'] -> whole milk, with support = 1.76% and confidence = 41.00%
10) ['whipped/sour cream'] -> whole milk, with support = 3.22% and confidence = 44.96%
11) ['white bread'] -> whole milk, with support = 1.71% and confidence = 40.58%
12) ['curd'] -> whole milk, with support = 2.61% and confidence = 49.05%
13) ['root vegetables'] -> whole milk, with support = 4.89% an

**Note:** The other case of this algorithm will be covered in a future project, using the `many to one` algorithm above.